TRABAJO GRUPAL PEL

Establecer los paths donde estan las librerias externas y cargarlas

In [1]:
#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libcurl")

#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libjsoncpp")

Incluir las librerias necesarias

In [2]:
#include <iostream>
#include <fstream>
#include <string>
#include <array>
#include <algorithm>

//Curl
#include <curl/curl.h>

//Json
#include <json/json.h>
#include <json/reader.h>
#include <json/writer.h>
#include <json/value.h>

//Mapas
#include "xleaflet/xmap.hpp"
#include "xleaflet/xtile_layer.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xmarker_cluster.hpp"
#include "xleaflet/xicon.hpp"
#include "xleaflet/xbasemaps.hpp"
#include "xleaflet/xpopup.hpp"
#include "xwidgets/xhtml.hpp"

//mapas y json
#include "xleaflet/xgeo_json.hpp"

using namespace std;


Estructura para las gasolineras

In [3]:
struct estacion {
    string provincia;
    string municipio;
    string localidad;
    string codigoPostal;
    string direccion;
    string margen;
    double longitud;
    double latitud;
    double precio;
    string rotulo;

    estacion *next;
};

Funcion para manejar la informacion de respuesta del GET REQUEST

In [4]:
static size_t WriteCallback(char* contenido, size_t size, size_t num, string* out) {
    out -> append(contenido, size * num);
        return size * num;
}

In [5]:
double toDouble(string s) {
    replace(s.begin(), s.end(), ',', '.');
        return atof(s.c_str());
}

In [6]:
static Json::Value ParseJson(string content) {
    Json::CharReaderBuilder builder;
    Json::CharReader* reader = builder.newCharReader();
    
    Json::Value jsonData;
    string errors;
    
    bool exitoParseando = reader->parse(content.c_str(), content.c_str() + content.size(), &jsonData, &errors);
    delete reader;
    
    if (!exitoParseando) {
        cout << "Error transfromando la informacion HTTP a JSON" << std::endl;
        cout << "Los datos del HTTP eran:\n" << content << std::endl;
        return 0;
    }
    else {
        cout << "Conversion de los datos a JSON exitosa" << endl;
        cout << "\ndatos del JSON recibidos" << endl;
        //cout << jsonData.toStyledString() << endl;
    
        const string dateString(jsonData["Fecha"].asString());


       std::cout << "\tDatos actualizados del: " << jsonData["Fecha"] << std::endl;
        std::cout << std::endl;
    }
    return jsonData;
    
}

Crear la lista

In [7]:
void getGasolineras(int tipo, estacion *&lista)
{
    string IDProducto;
    
    if(tipo == 1) {
        IDProducto = "1";
    }
    if(tipo == 2){
        IDProducto = "23";
    }
    if(tipo == 3){
        IDProducto = "20";
    }
    if (tipo == 4){
       IDProducto = "3";
    }
    if (tipo == 5){
        IDProducto = "21";
    }
    if (tipo == 6){
        IDProducto = "4";
    }
    if (tipo == 7){
        IDProducto = "5";    
    }
    if (tipo == 8){
        IDProducto = "6";
    }
    
    if (tipo < 9) {
    
    const std::string url("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/FiltroProducto/" + IDProducto);
    CURL* curl = curl_easy_init();
    string readBuffer;
    long httpCode(0);
    
    if (curl) {
        // Configurar como request GET
        curl_easy_setopt(curl, CURLOPT_CUSTOMREQUEST, "GET");
        // Configurar el URL.
        curl_easy_setopt(curl, CURLOPT_URL, url.c_str());
        // Para que no intente IPv6, porque aumentaria el tiempo de resolucion de DNS.
        curl_easy_setopt(curl, CURLOPT_IPRESOLVE, CURL_IPRESOLVE_V4);
        //Para que solo espere maximo 10 segundos la respuesta
        curl_easy_setopt(curl, CURLOPT_TIMEOUT, 10);
        // Follow HTTP redirects if necessary.
        curl_easy_setopt(curl, CURLOPT_FOLLOWLOCATION, 1L);
        
        // Enlazar la funcion que va a manejar la informacion de respuesta
        curl_easy_setopt(curl, CURLOPT_WRITEFUNCTION, WriteCallback);
        
        // Enlazar un contenedor para la informacion (will be passed as the last parameter to the
        // callback handling function).  
        curl_easy_setopt(curl, CURLOPT_WRITEDATA, &readBuffer);
        
        // Run our HTTP GET command, capture the HTTP response code, and clean up.
        curl_easy_perform(curl);
        curl_easy_getinfo(curl, CURLINFO_RESPONSE_CODE, &httpCode);
    }
   curl_easy_cleanup(curl);
    

    if (httpCode == 200)
    {
        std::cout << "\nGot successful response from " << url << std::endl;
        // La respuesta correcta - terminado con lo de curl.  
        // Intentar parsear los resultados e imprimirlos
        Json::StreamWriterBuilder builder;
        Json::Value jsonData = ParseJson(readBuffer);
        
        const Json::Value& listaprecio = jsonData["ListaEESSPrecio"];
    
        int i=0;
        while(i< listaprecio.size()) {
            estacion *gasolinera= new estacion();
            gasolinera->provincia = listaprecio[i]["Provincia"].asString();
            
            //Pasar a mayus
            string municipio = listaprecio[i]["Municipio"].asString();
            for_each(municipio.begin(), municipio.end(), [](char & c){
                c = ::toupper(c);
            });
            
            gasolinera->municipio = municipio;
            gasolinera->localidad = listaprecio[i]["Localidad"].asString();
            gasolinera->codigoPostal= listaprecio[i]["C.P."].asString();
            gasolinera->direccion= listaprecio[i]["Dirección"].asString();
            gasolinera->margen= listaprecio[i]["Margen"].asString();
            gasolinera->longitud = toDouble(listaprecio[i]["Longitud (WGS84)"].asString());
            gasolinera->latitud = toDouble(listaprecio[i]["Latitud"].asString());
            gasolinera->precio = toDouble(listaprecio[i]["PrecioProducto"].asString());
            gasolinera->rotulo= listaprecio[i]["Rótulo"].asString();
            
            if (lista==NULL){
                lista = gasolinera;
                gasolinera->next = NULL;
            }else{
                gasolinera->next = lista;
                lista = gasolinera;
            }
            //cout << "Direccion: " << listaprecio[i]["Dirección"].asString() <<endl;
            
            i++;
        }
        cout << "Numero de gasolineras: " << i << endl;
    }
    else
    {
        std::cout << "Couldn't GET from " << url << " - exiting" << std::endl;
        
    }
    }
}

estacion *header = NULL;

getGasolineras(1, header);

Buscar las gasolineras mas baratas

In [8]:
void gasolinera(estacion *list, string provincia, string municipio, string localidad){
    estacion *comprobar = list;
    estacion *header1 = list;
    estacion *peque1;
    estacion *header2 = list;
    estacion *peque2;
    estacion *header3 = list;
    estacion *peque3;
    estacion *header4 = list;
    estacion *peque4;
    estacion *header5 = list;
    estacion *peque5;
    int barato = 10000000;

    int contprovincia = 0;
    int contmunicipio = 0;
    int contlocalidad = 0;

    //comprobamos que los valores esten
    do{
        //cout << "Provincia actual: " << comprobar->provincia << endl;
        if(comprobar->provincia == provincia){
            contprovincia++;
        }
        if(comprobar->municipio == municipio){
            contmunicipio++;
        }
        if(comprobar->localidad == localidad){
            contlocalidad++;
        }
        comprobar = comprobar->next;
    }while(comprobar != NULL);
    
    cout << contprovincia << " contador2: " << contmunicipio << "contador3: " << contlocalidad << endl;
    if(contprovincia != 0 && contmunicipio != 0 && contlocalidad != 0) {
        cout<<"Buscando gasolineras localizacion indicada..."<<endl;
    }else if(contprovincia == 0 || contmunicipio == 0 || contlocalidad == 0){
        cout<<"Estos datos no se encuentran, asegurese de que estan correctamente escritos (Mayusculas)"<<endl;
        return;
    }

    double gasolina1, gasolina2, gasolina3, gasolina4, gasolina5 = header1->precio;
    float precio = 0;
    
    //para el mas barato
    do{
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && gasolina1<= barato && gasolina1!=0) {
            barato = gasolina1;
            peque1=header1;
            precio = peque1->precio;
        }
        header1 = header1->next;
    }while(header1!=NULL);

    if(precio == 0){
        cout<<"Ninguna gasolinera tiene este tipo de gasolina."<<endl;
        return;
    }
    cout<<"Calle: "<< peque1->direccion<<"   Precio: "<<precio<<endl;
    if(contlocalidad == 1){
        cout<<"En "<<localidad<<" solo existe una gasolinera."<<endl;
        return;
    }


    //para el segundo mas barato
    barato = 10000000;
    do{
        if((header2->provincia)==provincia && (header2->municipio)==municipio && (header2->localidad)==localidad && gasolina2<= barato && gasolina2!=0 && header2 != peque1) {
            barato = gasolina2;
            peque2=header2;
            precio = peque2->precio;
        }
        header2 = header2->next;
    }while(header2!=NULL);

    if(precio == 0){
        cout<<"Esta localidad solo tiene este tipo de gasolina una vez."<<endl;
        return;
    }
    cout<<"Calle: "<< peque2->direccion<<"   Precio: "<<precio<<endl;
    if(contlocalidad == 2){
        cout<<"En "<<localidad<<" solo existen dos gasolineras."<<endl;
        return;
    }

    //para el tercer mas barato
    barato = 10000000;
    do{
        if((header3->provincia)==provincia && (header3->municipio)==municipio && (header3->localidad)==localidad && gasolina3<= barato && gasolina3!=0 && header3 != peque2 && header3 != peque1) {
            barato = gasolina3;
            peque3=header3;
            precio = peque3->precio;
        }
        header3 = header3->next;
    }while(header3!=NULL);

    if(precio == 0){
        cout<<"Esta localidad solo tiene este tipo de gasolina dos veces."<<endl;
        return;
    }
    cout<<"Calle: "<< peque3->direccion<<"   Precio: "<<precio<<endl;
    if(contlocalidad == 3){
        cout<<"En "<<localidad<<" solo existen tres gasolineras."<<endl;
        return;
    }

    //para el cuarto mas barato
    barato = 10000000;
    do{
        if((header4->provincia)==provincia && (header4->municipio)==municipio && (header4->localidad)==localidad && gasolina4<= barato && gasolina4!=0 && header4 != peque3 && header4 != peque2 && header4 != peque1) {
            barato = gasolina4;
            peque4=header4;
            precio = peque4->precio;
        }
        header4 = header4->next;
    }while(header4!=NULL);

    if(precio == 0){
        cout<<"Esta localidad solo tiene este tipo de gasolina tres veces."<<endl;
        return;
    }
    cout<<"Calle: "<< peque4->direccion<<"   Precio: "<<precio<<endl;
    if(contlocalidad == 4){
        cout<<"En "<<localidad<<" solo existen cuatro gasolineras."<<endl;
        return;
    }

    //calculamos el pequeño 5
    barato = 10000000;
    do{
        if((header5->provincia)==provincia && (header5->municipio)==municipio && (header5->localidad)==localidad && gasolina5<= barato && gasolina5!=0 && header5 != peque4 && header5 != peque3 && header5 != peque2 && header5 != peque1) {
            barato = gasolina5;
            peque5=header5;
            precio = peque5->precio;
            
        }
        header5 = header5->next;
    }while(header5!=NULL);

    if(precio == 0){
        cout<<"Esta localidad solo tiene este tipo de gasolina cuatro veces."<<endl;
        return;
    }
    cout<<"Calle: "<< peque5->direccion<<"   Precio: "<<precio<<endl;
    if(contlocalidad == 5){
        cout<<"En "<<localidad<<" solo existen cinco gasolineras."<<endl;
        return;
    }

}

In [13]:

    estacion *header = NULL;

    string provinciaPedida, municipioPedido, localidadPedida;
    int tipoGasolina, repetir;

    //menu
    cout<<"  ACTIVIDAD GRUPAL PROGRAMACION CON ESTRUCUTRAS LINEALES  "<<endl;
    cout<<" La actividad consiste en realizar una busqueda de las 5 "<<endl;
    cout<<" gasolineras con los precios mas baratos en un tipo de "<<endl;
    cout<<" combustible de una localizacion indicada por el usuario."<<endl;
    cout<<" "<<endl;
    cout<<" Los participantes que han realizado este proyecto son: "<<endl;
    cout<<"     Pablo Pascual"<<endl;
    cout<<"     Miguel Ramos"<<endl;
    cout<<"     Victor Perez"<<endl;
    cout<<"     Nicolas Perez"<<endl;
    cout<<"     Maria Sol Torres"<<endl;
    cout<<"     Adilem Dobras"<<endl;
    cout<<"     Diego Martin"<<endl;
    cout<<" "<<endl;
    cout << "-------------------------------------------------" << endl;
    cout << "      Bienvenido al buscador de gasolineras" << endl;
    cout << "-------------------------------------------------" << endl;
    do{
        cout<<"Ingrese la provincia: "<<endl;
        cin>>provinciaPedida;
        cout<<"Ingrese el municipio: "<<endl;
        cin>>municipioPedido;
        cout<<"Ingrese la localidad: "<<endl;
        cin>>localidadPedida;
        cout<<"Que tipo gasolina necesita?"<<endl;
        cout<<" Tipos gasolina:  "<<endl;
        cout<<" 1/ Gasolina 95 E5"<<endl;
        cout<<" 2/ Gasolina 95 E10"<<endl;
        cout<<" 3/ Gasolina 95 E5 Premium"<<endl;
        cout<<" 4/ Gasolina 98 E5"<<endl;
        cout<<" 5/ Gasolina 98 E10"<<endl;
        cout<<" 6/ Gasoleo A"<<endl;
        cout<<" 7/ Gasoleo Premium"<<endl;
        cout<<" 8/ Gasoleo B"<<endl;
        cin>>tipoGasolina;

        getGasolineras(tipoGasolina, header);
        gasolinera(header,provinciaPedida,municipioPedido,localidadPedida);

        cout<<"Quiere buscar otro tipo de gasolina en una ubicacion diferente? (si=1 / no=0)"<<endl;
        cin>>repetir;
    }while(repetir == 1);
    if(repetir == 0){
        cout<"Hasta la proxima"<<endl;
    }

xlf::map mymap;
nodo *aux = localizaciones("MADRID", "MADRID");
mymap.zoom = 10;
// latitude 52.204793, longitude 360.121558
mymap.center = std::array<double, 2>({aux->latitud, aux->longitud});
mymap.display();

MAPA

A Jupyter widget

In [10]:
auto marker1 = xlf::marker::initialize()
    .location({(aux->latitud)+0.01, (aux->longitud)+0.01})
    .title(aux->municipio)
    .finalize();

In [11]:
mymap.add_layer(marker1);

mymap.display()

A Jupyter widget

In [12]:
xw::html message1;
xw::html message2;
message1.value = "Try clicking the marker!";
message2.value = "Hello <b>World</b>";
message2.placeholder = "Some HTML";
message2.description = "Some HTML";

// Popup with a given location on the map:
auto popup = xlf::popup::initialize()
    .location({aux->latitud, aux->longitud})
    .child(message1)
    .close_button(false)
    .auto_close(false)
    .close_on_escape_key(false)
    .finalize();
mymap.add_layer(popup);

// Popup associated to a layer
marker1.popup = message2;

In [ ]:
nodo *aux = localizaciones("MADRID", "MADRID");
xlf::marker marker;
int i=0;
array<xlf::marker, 5> array_markers;
    
        //cout << contenido << endl;
        while (aux != NULL && i<5) {
             std::cout << "en el while" <<std::endl;
             marker.location=std::array<double, 2>({aux->latitud, aux->longitud});
            array_markers[i] = marker;
            aux = aux->sgt;
            i++;
        }
    cout << array_markers.size() << endl;

xlf::marker_cluster marker_cluster = xlf::marker_cluster::initialize() .markers(array_markers)
.finalize();

mymap.add_layer(marker_cluster);

In [ ]:
nodo *aux = localizaciones("MADRID", "MADRID");
xlf::marker marker;
int i=0;
array<xlf::marker, 5> array_markers;
    
        //cout << contenido << endl;
        while (aux != NULL && i<5) {
             std::cout << "en el while" <<std::endl;
             marker.location=std::array<double, 2>({aux->latitud, aux->longitud});
            array_markers[i] = marker;
            aux = aux->sgt;
            i++;
        }
    cout << array_markers.size() << endl;

xlf::marker_cluster marker_cluster = xlf::marker_cluster::initialize() .markers(array_markers)
.finalize();

mymap.add_layer(marker_cluster);

In file included from input_line_9:2:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xhtml.hpp:13:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xstring.hpp:14:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xwidget.hpp:15:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xlayout.hpp:16:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xobject.hpp:18:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xwidgets/xtransport.hpp:19:
In file included from /Users/sol/opt/miniconda3/envs/cling/include/xproperty/xobserved.hpp:20:
/Users/sol/opt/miniconda3/envs/cling/include/xproperty/xproperty.hpp:201:19: error: no matching conversion for functional-style cast from
      'std::__1::array<xw::xmaterialize<xlf::xmarker>, 5>' to
      'xp::xproperty<std::__1::vector<xw::xholder,
      std::__1::allocator<xw::xholder> >, xw::xmaterialize<xlf:

In [44]:
mymap.add_layer()

A Jupyter widget